In [5]:
import numpy as np
wordsList = np.load('./training_data/wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('./training_data/wordVectors.npy')
print ('Loaded the word vectors!')

print(len(wordsList))
print(wordVectors.shape)

maxSeqLength = 250

ids = np.load('./training_data/idsMatrix.npy')
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0):
            num = randint(1,11499)
            labels.append([1,0])
        else:
            num = randint(13499,24999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels



batchSize = 64
lstmUnits = 64
numClasses = 2
numDimensions = 50 #Dimensions for each word vector
iterations = 10000

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)


lstmCell = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits)
lstmCell = tf.nn.rnn_cell.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

import datetime
sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    for i in range(iterations):
       #Next Batch of reviews
       nextBatch, nextBatchLabels = getTrainBatch();
       sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

       #Write summary to Tensorboard
       if (i % 10 == 0):
           summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
           writer.add_summary(summary, i)
           acc = sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})
           print('training! Step%d Training accuracy:%f'%(i,acc))

       #Save the network every 10,000 training iterations
       if (i % 10 == 0 and i != 0):
           save_path = saver.save(sess, "./models/pretrained_lstm.ckpt", global_step=i)
           print("saved to %s" % save_path)
    writer.close()


Loaded the word list!
Loaded the word vectors!
400000
(400000, 50)


/Users/alice/miniconda3/envs/deeplearning/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1752: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


training! Step0 Training accuracy:0.453125
training! Step10 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-10
training! Step20 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-20
training! Step30 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-30
training! Step40 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-40
training! Step50 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-50
training! Step60 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-60
training! Step70 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-70
training! Step80 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-80
training! Step90 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-90
training! Step100 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-100
training! Step110 Training accuracy:0.406250
saved to ./models/pretrained_lstm.ckpt-110
trainin

training! Step940 Training accuracy:0.437500
saved to ./models/pretrained_lstm.ckpt-940
training! Step950 Training accuracy:0.421875
saved to ./models/pretrained_lstm.ckpt-950
training! Step960 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-960
training! Step970 Training accuracy:0.390625
saved to ./models/pretrained_lstm.ckpt-970
training! Step980 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-980
training! Step990 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-990
training! Step1000 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-1000
training! Step1010 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-1010
training! Step1020 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-1020
training! Step1030 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-1030
training! Step1040 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-1040
training! Step1050 Tra

saved to ./models/pretrained_lstm.ckpt-1850
training! Step1860 Training accuracy:0.609375
saved to ./models/pretrained_lstm.ckpt-1860
training! Step1870 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-1870
training! Step1880 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-1880
training! Step1890 Training accuracy:0.640625
saved to ./models/pretrained_lstm.ckpt-1890
training! Step1900 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-1900
training! Step1910 Training accuracy:0.437500
saved to ./models/pretrained_lstm.ckpt-1910
training! Step1920 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-1920
training! Step1930 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-1930
training! Step1940 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-1940
training! Step1950 Training accuracy:0.593750
saved to ./models/pretrained_lstm.ckpt-1950
training! Step1960 Training accuracy:0.562500
saved to .

...

saved to ./models/pretrained_lstm.ckpt-5510
training! Step5520 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-5520
training! Step5530 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-5530
training! Step5540 Training accuracy:0.375000
saved to ./models/pretrained_lstm.ckpt-5540
training! Step5550 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-5550
training! Step5560 Training accuracy:0.609375
saved to ./models/pretrained_lstm.ckpt-5560
training! Step5570 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-5570
training! Step5580 Training accuracy:0.453125
saved to ./models/pretrained_lstm.ckpt-5580
training! Step5590 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-5590
training! Step5600 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-5600
training! Step5610 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-5610
training! Step5620 Training accuracy:0.593750
saved to .

training! Step6430 Training accuracy:0.656250
saved to ./models/pretrained_lstm.ckpt-6430
training! Step6440 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-6440
training! Step6450 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-6450
training! Step6460 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-6460
training! Step6470 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-6470
training! Step6480 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-6480
training! Step6490 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-6490
training! Step6500 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-6500
training! Step6510 Training accuracy:0.531250
saved to ./models/pretrained_lstm.ckpt-6510
training! Step6520 Training accuracy:0.453125
saved to ./models/pretrained_lstm.ckpt-6520
training! Step6530 Training accuracy:0.593750
saved to ./models/pretrained_lstm.ckpt-6530
training! 

saved to ./models/pretrained_lstm.ckpt-7340
training! Step7350 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-7350
training! Step7360 Training accuracy:0.531250
saved to ./models/pretrained_lstm.ckpt-7360
training! Step7370 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-7370
training! Step7380 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-7380
training! Step7390 Training accuracy:0.531250
saved to ./models/pretrained_lstm.ckpt-7390
training! Step7400 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-7400
training! Step7410 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-7410
training! Step7420 Training accuracy:0.609375
saved to ./models/pretrained_lstm.ckpt-7420
training! Step7430 Training accuracy:0.453125
saved to ./models/pretrained_lstm.ckpt-7430
training! Step7440 Training accuracy:0.484375
saved to ./models/pretrained_lstm.ckpt-7440
training! Step7450 Training accuracy:0.656250
saved to .

training! Step8260 Training accuracy:0.453125
saved to ./models/pretrained_lstm.ckpt-8260
training! Step8270 Training accuracy:0.640625
saved to ./models/pretrained_lstm.ckpt-8270
training! Step8280 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-8280
training! Step8290 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-8290
training! Step8300 Training accuracy:0.656250
saved to ./models/pretrained_lstm.ckpt-8300
training! Step8310 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-8310
training! Step8320 Training accuracy:0.562500
saved to ./models/pretrained_lstm.ckpt-8320
training! Step8330 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-8330
training! Step8340 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-8340
training! Step8350 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-8350
training! Step8360 Training accuracy:0.625000
saved to ./models/pretrained_lstm.ckpt-8360
training! 

saved to ./models/pretrained_lstm.ckpt-9170
training! Step9180 Training accuracy:0.593750
saved to ./models/pretrained_lstm.ckpt-9180
training! Step9190 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-9190
training! Step9200 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-9200
training! Step9210 Training accuracy:0.468750
saved to ./models/pretrained_lstm.ckpt-9210
training! Step9220 Training accuracy:0.578125
saved to ./models/pretrained_lstm.ckpt-9220
training! Step9230 Training accuracy:0.500000
saved to ./models/pretrained_lstm.ckpt-9230
training! Step9240 Training accuracy:0.453125
saved to ./models/pretrained_lstm.ckpt-9240
training! Step9250 Training accuracy:0.515625
saved to ./models/pretrained_lstm.ckpt-9250
training! Step9260 Training accuracy:0.546875
saved to ./models/pretrained_lstm.ckpt-9260
training! Step9270 Training accuracy:0.593750
saved to ./models/pretrained_lstm.ckpt-9270
training! Step9280 Training accuracy:0.578125
saved to .

In [4]:
print(logdir)

tensorboard/20200321-131533/


In [ ]:
import numpy as np
wordsList = np.load('./training_data/wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('./training_data/wordVectors.npy')
print ('Loaded the word vectors!')

print(len(wordsList))
print(wordVectors.shape)

maxSeqLength = 250

ids = np.load('./training_data/idsMatrix.npy')
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        if (i % 2 == 0):
            num = randint(1,11499)
            labels.append([1,0])
        else:
            num = randint(13499,24999)
            labels.append([0,1])
        arr[i] = ids[num-1:num]
    return arr, labels



batchSize = 24
lstmUnits = 64
numClasses = 2
numDimensions = 50 #Dimensions for each word vector
iterations = 380000

# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
#############################################################
lstmCell = tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True)
lstmCell = tf.nn.rnn_cell.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
##########################################################################
# cell_list = []
# for _ in range(2):
#     lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits, state_is_tuple=True)
#     cell_list.append(lstmCell)
# lstmCell=tf.contrib.rnn.MultiRNNCell(cell_list, state_is_tuple=True)
###############################################################################


initial_state = lstmCell.zero_state(batchSize, tf.float32)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, initial_state=initial_state,dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
tf.summary.scalar('weights1', weight[1,1])
tf.summary.scalar('weights2', weight[3,1])
tf.summary.histogram("weights",weight)

bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
tf.summary.histogram("biases",bias)
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(0.0005).minimize(loss)

import datetime
sess = tf.InteractiveSession()
tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
# logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
logdir="./mygraph/"
writer = tf.summary.FileWriter(logdir, sess.graph)

# gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
# config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
with tf.Session(config=config) as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, './models3/pretrained_lstm.ckpt-80000')

    for i in range(300000,iterations):
       #Next Batch of reviews
       nextBatch, nextBatchLabels = getTrainBatch();
       sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})

       #Write summary to Tensorboard
       if (i % 50 == 0):
           summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
           writer.add_summary(summary, i)
           l,acc= sess.run([loss,accuracy], {input_data: nextBatch, labels: nextBatchLabels})
           print('training! Step%d Training loss: %f  Training acc:%f'%(i,l,acc))

       #Save the network every 10,000 training iterations
       if (i % 10000 == 0 and i != 0):
           save_path = saver.save(sess, "./training_process_model2/pretrained_lstm.ckpt", global_step=i)
           print("saved to %s" % save_path)
    writer.close()
